## 実践タスク: 琵琶湖の低雲量画像抽出
1. 必要な情報の準備

|          項目      |   値                               |       目的                 |
|-------------------|------------------------------------|---------------------------|
|データセットID       |COPERNICUS/S2_SR                    |Sentinel-2 表面反射率データ   |
|対象期間            |2025年9月1日 〜 2025年11月1日          |過去2ヶ月間の設定             |
|雲量上限            |5%                                  |低雲量画像のみを抽出           |
|対象地域 (琵琶湖)    |ee.Geometry.Point([136.17, 35.10])  |緯度・経度 (中心付近)          |

2. pythonコードによる実践

In [2]:
import ee

In [3]:
import datetime

In [4]:
ee.Initialize(project='earth-change-analysis')

In [5]:
## 1. 地域の定義 (琵琶湖の中心点)
biwako_point = ee.Geometry.Point([136.17, 35.10])

In [6]:
## 2. 日付の定義 (現在の応答時間に基づき2025年を設定)
# 開始日: 2ヶ月前
start_date = (datetime.date(2025, 11, 15) - datetime.timedelta(days=60)).strftime('%Y-%m-%d')

In [7]:
# 終了日: 今日
end_date = '2025-11-15'

### 今回は簡略化のため、直接日付を設定
- start_date = '2025-09-01'
- end_date = '2025-11-01'

In [8]:
print(f"検索期間: {start_date} から {end_date}")

検索期間: 2025-09-16 から 2025-11-15


In [9]:
## 3. 画像コレクションのロード
s2_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')

#### Sentinel-2 SR Harmonized コレクションを取得

In [10]:
## 4. フィルタリングの実行 (3つのフィルターを連結)
filtered_collection = s2_collection.filterDate(start_date, end_date)\
    .filterBounds(biwako_point)\
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 5)

In [11]:
## 5. 結果の確認
# 抽出された画像の総数を取得
count = filtered_collection.size().getInfo()
print(f"抽出された画像の総数: {count} 枚")

抽出された画像の総数: 1 枚


In [12]:
# 抽出されたコレクションから最初の画像の日付と雲量プロパティを確認
if count > 0:
    first_image = filtered_collection.first()
    
    # 日付の確認
    date_ms = first_image.get('system:time_start').getInfo()
    date_dt = datetime.datetime.fromtimestamp(date_ms / 1000).strftime('%Y-%m-%d')
    
    # 雲量の確認
    cloud_cover = first_image.get('CLOUDY_PIXEL_PERCENTAGE').getInfo()
    
    print(f"\n最初の画像の情報:")
    print(f"  - 撮影日: {date_dt}")
    print(f"  - 雲量 (%): {cloud_cover:.2f}%")
else:
    print("\n指定された条件に一致する画像は見つかりませんでした。")    


最初の画像の情報:
  - 撮影日: 2025-11-07
  - 雲量 (%): 4.85%


---
- count > 0
→ フィルタ後の画像コレクションに1枚以上画像があるか確認しています。
- filtered_collection.first()
→ コレクションの中で最初の画像（最も早い日付のもの）を取得します
---

---
- system:time_start
→ 画像が撮影された日時（ミリ秒単位のUNIXタイムスタンプ）を持つメタデータ。
- .getInfo()
→ Earth Engine のオブジェクトを Python の値に変換。ここでは整数値（ミリ秒）になります。
- datetime.datetime.fromtimestamp(date_ms / 1000)
→ ミリ秒を秒に直して Python の日時型に変換。
- .strftime('%Y-%m-%d')
→ 「YYYY-MM-DD」形式の文字列に整形。
- 結果: 撮影日が「2020-01-15」などの形で得られます。
---

---
- CLOUDY_PIXEL_PERCENTAGE
→ 第一引数：画像全体に占める「雲の割合 (%)」を示すメタデータキー。
- .getInfo()
→ Python の数値として取得。例えば 12.34 のようにパーセンテージが返ります。
---

.filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 5):

これが雲量フィルターです。

第1引数: フィルタリングの対象となるメタデータキー (CLOUDY_PIXEL_PERCENTAGE はSentinel-2の雲量プロパティ)。

第2引数: 演算子 (less_than は「より小さい」を意味します)。

第3引数: 値 (5、つまり5%未満)。

---
- f"..." は f-string（フォーマット文字列）。変数を埋め込んで表示できます。
- :.2f は小数点以下2桁まで表示する指定。
---

## 初学者向けまとめ
- 目的: フィルタした画像コレクションから「最初の画像の日付と雲量」を確認する。
- ポイント:
- .get('system:time_start') → 撮影日時
- .get('CLOUDY_PIXEL_PERCENTAGE') → 雲の割合
- .getInfo() → Earth Engine の値を Python に変換
- datetime → 人間が読める日付に変換
